# Smart Home Energy Management 
First attempt at getting a working model. 

In [23]:
using POMDPs
using POMDPToolbox
using POMDPModelTools
using Random, Distributions

rng = MersenneTwister(1234) # TODO: Make each instance of 'rng' a different seed 

MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

## Global Model Parameters 

First, try to get an extremely basic version of the model running. See SIMPLE MODEL DATA PARAMS for details 

### Real Model Data
Need to write function to read in arrays of all the load/solar, occupancy, etc. from CSV files 

In [24]:
# Load Real Data Model Params 

# Import Data from CSV 
function import_data(fileapth)
    # TODO: Write import function 
    # return D, OCC, TOD, t, ODT, TOU
end


# Battery Params 
SOC_MAX = 13500; # Wh 
C_MAX = 5000; # Wh/hr 
C_RES = 100 # Wh (granularity of space) 

# Thermal Params 
TCMFT_LO = 67 # Lower thermal comfort band temp 
TCMFT_HI = 71 # Upper thermal comfort band temp 

# Building Params 
D_MAX = 5000; # Wh/hr 
D_RES = 100 # Wh (granularity of space) 


100

### SIMPLE Model Data

In [25]:
# Simple Model Params 

D_ = [3,5,3,9,8,7,5,4,3,2]; # demand (non-hvac) 
OCC = [1,0,0,1,1,1,0,0,0,1]; # occupancy 
TOD = [1,2,3,4,5,1,2,3,4,5]; # time of day 
t = collect(1:10); # time index 
ODT = [3,2,3,6,5,4,3,2,4,3]; # outdoor temp 
TOU = [2,2,3,4,2,2,2,3,4,2]; # time of use rate 


## States
Data container representing the state of the smarthome 

In [26]:
struct SmartHomeState
    d_hv::Int64 # demand of hvac (Wh, rounded to nearest 100Wh) 
    d_::Int64 # demand of all other loads (Wh, rounded to nearest 100Wh)
    soc::Int64 # state of charge of battery (Wh, rounded to nearest 100 Wh) 
    rmt::Int64 # room temperature (degF, to nearest degree) 
    occ::Bool # occupancy status 
    hsp::Int64 # heating setpoint (degF, to nearest degree) 
    csp::Int64 # cooling setpoint (degF, to nearest degree) 
    tod::Int64 # time of day (hr, 0-23) 
    odt::Int64 # outdoor temp (degF, to nearest degree) 
    tou::Int64 # time of use rate ($0.01, to nearest cent) 
    t::Int64 # current time index 
end

### TODO: Initalize State
 


Convenience functions for working with the SmartHome state 

In [27]:
# initial state constructor 
SmartHomeState(d_hv::Int64, d_::Int64, rmt::Int64, occ::Bool, hsp::Int64, csp::Int64, tod::Int64, odt::Int64, tou::Int64) = SmartHomeState(d_hv, d_, 0.5*SOC_MAX, rmt, occ, hsp, csp, tod, odt, tou)

# checks if the setpoints of two states are the same 
spequal(s1::SmartHomeState, s2::SmartHomeState) = s1.hsp == s2.hsp && s1.csp == s2.csp 

spequal (generic function with 1 method)

## Actions
Actions are defined by both the charge rate of tha battery and the change in thermostat setpoint 

In [28]:
struct SmartHomeAction
    c::Int64 # Charge rate of battery (Wh, to nearest 100 Wh) 
    dhsp::Int64 # Change in heat setpoint (degF, to nearest degree) 
    dcsp::Int64 # Change in cool setpoint (degF, to nearest degree) 
end

## MDP 
SmartHome data container is defined. It holds all the information needed to define the MDP tuple (S, A, T, R). 


In [37]:
# the smarthome mdp type 
struct SmartHomeMDP <: MDP{SmartHomeState, SmartHomeAction} 
    # Note that our MDP is parametrized by the state and the action
    # Given Data 
    D_::Array{Int64}
    OCC::Array{Bool}
    TOD::Array{Int64}
    t::Array{Int64}
    ODT::Array{Int64}
    TOU::Array{Int64} 
    
    # Battery Params
    soc_max::Int64 
    c_max::Int64
    penalty_soc::Float64
    
    # Comfort Params 
    tcomf_lo::Int64
    tcomf_hi::Int64
    prob_sp_adj::Float64
    penalty_discomf::Float64 
    
    # Building Params 
    d_max::Int64 
    penalty_sp::Float64 
end

SmartHomeMDP() = SmartHomeMDP(D_, OCC, TOD, t, ODT, TOU, 10, 5, -1000, 4, 6, 0.3, -1, 10, -1000); 

### TODO: Define Action Space 
Define the action space c=collect(-5:5), dhsp=collect(-5:5), dcsp=collect(-5:5)

In [44]:
POMDPs.actions(::SmartHomeMDP) = SmartHomeAction(
    collect(-5:5), # Charge rate of battery 
    collect(-5:5), # Change in heat setpoint
    collect(-5:5)  # Change in cool setpoint
)

## Define gen 
Implement the complete generative models for both the SmartHomeMDP and SmartHomePOMDP. 

### State transition model 
- d_hv: f(hsp, csp, odt) 
- d_: probabilistic f(d_(t-1), tod, rand)
- soc: f(soc(t-1), c(t-1)) 
- rmt: f(hsp, csp, odt) 
- occ: probabilistic f(tod, rand) 
- hsp: f(hsp(t-1), dhsp, rand) 
- csp: f(csp(t-1), dcsp, rand) 
- tod: f(tod(t-1))
- odt: f(odt(t-1), tod) 
- tou: f(tod) 

### Observation model 
- occ_d: f(occ, p_tp, p_fp) 
    - p_tp: true pos prob, 
    - p_fp: false pos prob
    
### Reward model
Reward model is composed of the following components: 
- TOU charge demand_: f(tou, d_) 
- TOU charge on demand_hvac: f(tou, d_hv) 
- TOU charge on charge_rate: f(tou, c) 
- Discomfort Penalty: f(occ, rmt) 
- SOC violation Penalty: f(soc)  
- SP violation penalty: f(hsp, csp) 
- Demand Charge: f(D_, D_HV, C) depends on all timesteps for the entire duration 


In [31]:
function update_sps(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    if a.dhsp != 0 || a.dcsp != 0 
        # If action=adjust, deterministically set thermostat 
        return s.hsp + a.dhsp, s.csp + a.dcsp
    else
        # Else, probabilistic adjustment based on thermal comfort 
        if rand(rng) < m.prob_sp_adj
            # Adjust setpoints 
            septoints = [round(x) for x in rand(rng, Uniform(m.tcomf_lo, m.tcomf_hi),2)]
            return min(setpoints), max(setpoints) 
        else
            # Maintain previous SPs
            return s.hsp, s.csp 
        end
    end
end

function hv_model(hsp, csp, odt, rng) 
    # Fitted regression model based hv_model data analysis. 
    # Model was fit to predict kWh/day 
    # d_hv = a*cdd + b*hdd + c + N(0,RMSE)  
    
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 2.1898; b = 0.9476; c = 0.5394; RMSE = 6.146; 
    error = rand(rng, Normal(0,RMSE), 1) 
    
    d_hv = a*cdd + b*hdd + c + error
    d_hv = round(d_hv*1000/24, digits=-2) # kWh/day -> Wh/hr(to nearest 100Wh) 
    
    return d_hv
end

function hv_model_simple(hsp, csp, odt, rng)
    # Extremely simplified model to work with SIMPLE DATA 
    cdd = max(odt-csp,0) # cdd in an hour 
    hdd = max(hsp-odt,0) # hdd in an hour 
    
    a = 2.3; b = 1.7; RMSE = 0.8; 
    error = rand(rng, Normal(0,RMSE), 1) 
    
    d_hv = a*cdd + b*hdd + error 
    d_hv = max(min(round(d_hv),10), 0) 
    
    return d_hv
end

function update_d_(d_, tod)
    # TODO: fit model dependent on TOD
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    noise_d_ = 3
    
    d_ += rand(rng, Normal(0,noise_d_), 1) # Probabilistic Change 
    d_ = max(min(round(d_),10), 0) # Clamp to 0-10 int
    return d_
end

function update_occ(occ, tod)
    # TODO: fit model dependent on TOD 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # Maybe fit a BN to this data? 
    p_change_occ = 0.3
    
    return rand(rng) < p_change_occ ? !occ : occ
end

function update_odt(odt, tod)
    # Fitted to a sine wave with noise 
    # TODO: need to adjust equation params when converting to realistic data 
    # TODO: adjust clamp when realistic data (maybe functionalize clamp) 
    # TODO: fit model dependent on TOD 
    # Maybe fit a BN to this data? 
    noise_odt = 1
    
    odt = 2.5*sin((tod+1.75)*pi/2.5)+5
    odt += rand(rng, Normal(0,noise_odt), 1) 
    odt = max(min(round(odt),10), 0) # Clamp to 0-10 int 
    
    return odt
end

function update_tou(tod) 
    TOU_SCHEDULE = [2,2,3,4,2]; 
    # TODO: Change tou_schedule to take as model input rather than hard code 
    tou = TOU_SCHEDULE[tod] 
    return tou 
end


# MDP Generative Model 
function POMDPs.gen(m::SmartHomeMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    t = s.t + 1 # Deterministic, fixed 
    tod = rem(s.tod + 1, 24) # Deterministic, fixed 
    
    #d_ = m.D_[s.t+1] # Deterministic, fixed (should/could it be probabilistic?)
    #occ = OCC[s.t+1] # Deterministic, fixed (should/could it be probabilistic?) 
    #odt = ODT[s.t+1] # Deterministic, fixed 
    #tou = TOU[s.t+1] # Deterministic, fixed 
    
    d_ = update_d_(s.d_, tod) # Probabilistic
    occ = update_occ(s.occ, tod) # Probabilistic
    odt = update_odt(tod) # Probabilistic
    tou = update_tou(tod) # By Schedule
    
    
    (hsp, csp) = update_sps(m, s, a, rng) 
    #d_hv = hv_model(hsp, csp, odt, rng) # Based on actual fit 
    d_hv = hv_model_simple(hsp, csp, s.odt, rng) 
    soc = s.soc + a.c
    rmt = min(max(hsp,odt),csp)
    
    sp = SmartHomeState(d_hv, d_, soc, rmt, occ, hsp, csp, tod, odt, tou, t)
    

    # observation model 
    # N/A
    
    # reward model 
    r = tou * (s.d_ + s.d_hv + a.c)
    r += (s.rmt > m.tcomf_hi || s.rmt < m.tcomf_lo) ? m.penalty_discomf : 0 
    r += (s.soc > m.soc_max || s.soc < 0) ? m.penalty_soc : 0 
    r += (s.hsp > s.csp) ? m.penalty_sp : 0 # HSP must be less than or equal to CSP 
    
    
    # create and return a NamedTuple 
    return (sp=sp, r=r) # For MDP 
end



### TODO: Convert above code from MDP -> POMDP

In [32]:
# POMDP Generative Model 
function POMDPs.gen(m::SmartHomePOMDP, s::SmartHomeState, a::SmartHomeAction, rng) 
    # transition model 
    # should be same as MDP 
    
    # observation model 
    if sp # Opccupied 
        o = rand(rng) < m.p_occ_tp
    else # Not Occupied 
        o = rand(rng) < m.p_occ_fp
    end
    
    
    # reward model 
    
    
    # create and return a NamedTuple 
    return (sp=sp, o=o, r=r) # For POMDP 
end



UndefVarError: UndefVarError: SmartHomePOMDP not defined

## Initial State Distribution
At first, lets assume a somewhat arbitrary assignment of deterministic states and assume the transition model will handle the rest. 


In [33]:
#POMDPs.initialstate_distribution(m::BabyPOMDP) = Deterministic(false)
POMDPs.initialstate_distribution(m::SmartHomeMDP) = SmartHomeState(
    Deterministic(5), # d_hv 
    Deterministic(5), # d_
    Deterministic(5), # soc 
    Deterministic(5), # rmt
    Deterministic(true), # occ
    Deterministic(4), # hsp
    Deterministic(6), # csp
    Deterministic(1), # tod
    Deterministic(5), # odt
    Deterministic(2), # tou
    Deterministic(1) # t 
) 

POMDPs.initialstate_distribution(m::SmartHomeMDP) = SmartHomeState(5, 5, 5, 5, t) 

In [45]:
sh = SmartHomeMDP()

SmartHomeMDP([3, 5, 3, 9, 8, 7, 5, 4, 3, 2], Bool[1, 0, 0, 1, 1, 1, 0, 0, 0, 1], [1, 2, 3, 4, 5, 1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [3, 2, 3, 6, 5, 4, 3, 2, 4, 3], [2, 2, 3, 4, 2, 2, 2, 3, 4, 2], 10, 5, -1000.0, 4, 6, 0.3, -1.0, 10, -1000.0)

## Step Through Random Policy 

In [51]:
using POMDPSimulators
using POMDPPolicies



UndefVarError: UndefVarError: m not defined

In [52]:
policy = RandomPolicy(sh)

for (s, a, r) in stepthrough(sh, policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
    println()
end

MethodError: MethodError: Cannot `convert` an object of type Deterministic{Int64} to an object of type Int64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Integer}, !Matched::Ptr) where T<:Integer at pointer.jl:23
  ...